In [7]:
import time
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# 加载数据，第一次需要下载，之后就不需要再下载
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
# 一个简单的卷积神经网络
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
# 训练过程
def train(net, lr, batch):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
    last_loss = 0.0
    # 多批次循环
    for epoch in range(batch):  

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # 获取输入
            inputs, labels = data

            # 梯度置0
            optimizer.zero_grad()

            # 正向传播，反向传播，优化
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # 打印状态信息
            running_loss += loss.item()
            if i % 2000 == 1999:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                last_loss = running_loss / 2000
                running_loss = 0.0

    print('Finished Training. The last loss is: %.3f' % last_loss)
    return last_loss

In [5]:
net = Net()
print(train(net, 0.01, 20))

[1,  2000] loss: 2.078
[1,  4000] loss: 1.970
[1,  6000] loss: 1.933
[1,  8000] loss: 1.954
[1, 10000] loss: 1.951
[1, 12000] loss: 1.953
[2,  2000] loss: 1.957
[2,  4000] loss: 1.970
[2,  6000] loss: 1.959
[2,  8000] loss: 1.989
[2, 10000] loss: 1.965
[2, 12000] loss: 2.024
[3,  2000] loss: 1.968
[3,  4000] loss: 1.986
[3,  6000] loss: 2.017
[3,  8000] loss: 2.001
[3, 10000] loss: 1.994
[3, 12000] loss: 2.015
[4,  2000] loss: 2.016
[4,  4000] loss: 1.991
[4,  6000] loss: 2.000
[4,  8000] loss: 1.977
[4, 10000] loss: 1.991
[4, 12000] loss: 2.003
[5,  2000] loss: 1.984
[5,  4000] loss: 1.994
[5,  6000] loss: 1.989
[5,  8000] loss: 1.967
[5, 10000] loss: 1.994
[5, 12000] loss: 1.979
[6,  2000] loss: 1.976
[6,  4000] loss: 1.980
[6,  6000] loss: 1.959
[6,  8000] loss: 2.013
[6, 10000] loss: 1.975
[6, 12000] loss: 2.029
[7,  2000] loss: 1.984
[7,  4000] loss: 2.017
[7,  6000] loss: 1.999
[7,  8000] loss: 1.989
[7, 10000] loss: 1.981
[7, 12000] loss: 1.979
[8,  2000] loss: 2.057
[8,  4000] 

In [5]:
# 测试集
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))
    return 100 * correct / total

In [8]:
params = []
lr_list = [0.1, 0.01, 0.001, 0.0001]
batch_num = [5, 10, 15, 20]
last_loss = []
accuracy = []
for lr in lr_list:
    for batch in batch_num:
        print('learning rate is %.4f, batch num is %d' % (lr, batch))
        net = Net()
        last_loss.append(train(net, lr, batch))
        accuracy.append(test(net))
        time.sleep(60)

learning rate is 0.1000, batch num is 5
[1,  2000] loss: 2.357
[1,  4000] loss: 2.359
[1,  6000] loss: 2.357
[1,  8000] loss: 2.364
[1, 10000] loss: 2.362
[1, 12000] loss: 2.360
[2,  2000] loss: 2.360
[2,  4000] loss: 2.360
[2,  6000] loss: 2.357
[2,  8000] loss: 2.358
[2, 10000] loss: 2.359
[2, 12000] loss: 2.359
[3,  2000] loss: 2.360
[3,  4000] loss: 2.358
[3,  6000] loss: 2.364
[3,  8000] loss: 2.360
[3, 10000] loss: 2.356
[3, 12000] loss: 2.358
[4,  2000] loss: 2.355
[4,  4000] loss: 2.365
[4,  6000] loss: 2.355
[4,  8000] loss: 2.358
[4, 10000] loss: 2.361
[4, 12000] loss: 2.362
[5,  2000] loss: 2.363
[5,  4000] loss: 2.361
[5,  6000] loss: 2.357
[5,  8000] loss: 2.363
[5, 10000] loss: 2.359
[5, 12000] loss: 2.360
Finished Training. The last loss is: 2.360
Accuracy of the network on the 10000 test images: 10 %
learning rate is 0.1000, batch num is 10
[1,  2000] loss: 2.357
[1,  4000] loss: 2.358
[1,  6000] loss: 2.360
[1,  8000] loss: 2.360
[1, 10000] loss: 2.359
[1, 12000] loss:

In [9]:
last_loss

[2.3595737093687057,
 2.3582609359622,
 2.3623102653622627,
 2.3624712008833884,
 1.985755041897297,
 1.9783041796088219,
 1.9672023037672042,
 2.3077288702726366,
 1.0262536453967914,
 0.8985468287020921,
 0.7607608961760998,
 0.6970488857435485,
 1.4471023710891604,
 1.1631816812232136,
 1.010530238211155,
 0.8914304749961011]

In [10]:
accuracy

[10.0,
 10.0,
 10.0,
 10.0,
 23.84,
 25.15,
 24.16,
 10.0,
 59.06,
 62.51,
 61.83,
 61.41,
 46.75,
 56.28,
 62.48,
 63.5]